In [ ]:
import cv2
import numpy as np
from action_package import ROBOT
import RPi.GPIO as GPIO
import time
import threading
import ipywidgets.widgets as widgets
from IPython.display import display
from collections import deque

# Global state variable
action_in_progress = threading.Event()  # Use event objects to mark whether an action is being performed

# Initializing robot
robot = ROBOT()

speed = 40

# Obstacle avoidance sensor configuration
SensorRight = 16
SensorLeft = 12
TRIG = 20
ECHO = 21

# Camera configuration
cam = cv2.VideoCapture(0, cv2.CAP_V4L2)
cam.set(3, 320)  # width
cam.set(4, 240)  # height

# Obstacle avoidance correlation function
def setup_sensors():
    GPIO.setwarnings(False)
    GPIO.setmode(GPIO.BCM)
    GPIO.setup(SensorRight, GPIO.IN)
    GPIO.setup(SensorLeft, GPIO.IN)
    GPIO.setup(TRIG, GPIO.OUT)
    GPIO.setup(ECHO, GPIO.IN)

def get_distance():
    """Get the distance of the ultrasonic sensor"""
    GPIO.output(TRIG, 0)
    time.sleep(0.000002)
    GPIO.output(TRIG, 1)
    time.sleep(0.00001)
    GPIO.output(TRIG, 0)

    while GPIO.input(ECHO) == 0:
        pass
    start_time = time.time()

    while GPIO.input(ECHO) == 1:
        pass
    end_time = time.time()

    distance = (end_time - start_time) * 340 / 2 * 100
    time.sleep(0.02)
    return round(distance, 2)

# Convert image to JPEG format
def bgr8_to_jpeg(value, quality=75):
    return bytes(cv2.imencode('.jpg', value)[1])

# Initializes the display area
FGmaskComp_img = widgets.Image(format='jpeg', width=320, height=240)
frame_img = widgets.Image(format='jpeg', width=320, height=240)
display(widgets.HBox([FGmaskComp_img, frame_img]))

# Obstacle avoiding thread
obstacle_event = threading.Event()
gesture_event = threading.Event()

def obstacle_avoidance():
    """Obstacle avoidance logic"""
    setup_sensors()
    while not obstacle_event.is_set():
        dist = get_distance()
        SR = GPIO.input(SensorRight)
        SL = GPIO.input(SensorLeft)

        if dist < 10 or SR == 0 or SL == 0:
            gesture_event.set()  # Pause gesture detection
            action_in_progress.set()  # Set the status to busy

            robot.t_down(speed, 1)  # Back 1 second
            robot.t_stop(0)      # stop
            time.sleep(0.5)      # Delay to avoid miscontact

            if SL == 0:
                robot.turnRight(speed, 0.5)  # Turn right for 0.5 seconds
                robot.t_stop(0)
            elif SR == 0:
                robot.turnLeft(speed, 0.5)  # Turn left 0.5 seconds
                robot.t_stop(0)

            action_in_progress.clear()  # Clear busy state
            gesture_event.clear()  # Recovery gesture detection
        time.sleep(0.1)

# Gesture control thread
def detect_gesture(mask, frame):
    """gesture recognition"""
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours = sorted(contours, key=lambda x: cv2.contourArea(x), reverse=True)
    
    for cnt in contours:
        area = cv2.contourArea(cnt)
        if area < 500:  # Filter for smaller Outlines
            continue
        
        (x, y, w, h) = cv2.boundingRect(cnt)
        cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 3)
        
        hull = cv2.convexHull(cnt, returnPoints=False)
        if len(hull) < 3:
            continue
        
        try:
            defects = cv2.convexityDefects(cnt, hull)
            if defects is not None:
                finger_count = 0
                for i in range(defects.shape[0]):
                    s, e, f, d = defects[i, 0]
                    start = tuple(cnt[s][0])
                    end = tuple(cnt[e][0])
                    far = tuple(cnt[f][0])

                    # calculate angle
                    a = np.linalg.norm(np.array(start) - np.array(far))
                    b = np.linalg.norm(np.array(end) - np.array(far))
                    c = np.linalg.norm(np.array(start) - np.array(end))
                    angle = np.arccos((a**2 + b**2 - c**2) / (2 * a * b)) * 57

                    if angle <= 90:  # Finger detected
                        finger_count += 1
                        cv2.line(frame, start, far, [0, 255, 0], 2)
                        cv2.line(frame, end, far, [0, 255, 0], 2)
                
                if finger_count == 1:
                    return "Left"
                elif finger_count == 2:
                    return "Right"
                elif finger_count == 3:
                    return "Forward"
                elif finger_count == 4:
                    return "Backward"
                elif finger_count == 5:
                    return "Stop"
        except cv2.error as e:
            print(f"Error detecting convexity defects: {e}")
    
    return None

def gesture_control():
    last_gesture = None
    last_gesture_time = time.time()  # Record the last gesture recognition time
    gesture_delay = 5  # Set the delay time between gesture recognition (seconds)

    while True:
        if gesture_event.is_set():  # Obstacle avoidance logic is running
            time.sleep(0.1)
            continue

        if action_in_progress.is_set():  # Action not completed, wait until complete to continue
            time.sleep(0.1)
            continue

        ret, frame = cam.read()
        if not ret:
            print("Failed to capture frame from camera")
            continue

        frame = cv2.flip(frame, 0)
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

        # Color range (skin tone range, palm color) :
        lower_skin = np.array([0, 20, 70], dtype=np.uint8)  # Lower bound for skin color
        upper_skin = np.array([20, 255, 255], dtype=np.uint8)  # Upper bound for skin color
        mask = cv2.inRange(hsv, lower_skin, upper_skin)

        FGmaskComp_img.value = bgr8_to_jpeg(mask)
        gesture = detect_gesture(mask, frame)

        # Check whether the gesture recognition delay condition is met
        current_time = time.time()
        if gesture and gesture != last_gesture and (current_time - last_gesture_time) >= gesture_delay:
            action_in_progress.set()  # Mark busy state

            if gesture == "Left":
                robot.moveLeft(speed, 1)
            elif gesture == "Right":
                robot.moveRight(speed, 1)
            elif gesture == "Forward":
                robot.t_up(speed, 3)
            elif gesture == "Backward":
                robot.t_down(speed, 3)
            elif gesture == "Stop":
                robot.t_stop(0)

            # Stop when the action is complete
            robot.t_stop(0)
            action_in_progress.clear()  # Clear the busy state and prepare for the next action
            last_gesture = gesture
            last_gesture_time = current_time  # Update the time of the last recognition

        frame_img.value = bgr8_to_jpeg(frame)  # Update Display

    cam.release()


# Mainline start
try:
    obstacle_thread = threading.Thread(target=obstacle_avoidance)
    gesture_thread = threading.Thread(target=gesture_control)

    obstacle_thread.start()
    gesture_thread.start()

    obstacle_thread.join()
    gesture_thread.join()
except KeyboardInterrupt:
    obstacle_event.set()
    gesture_event.set()
    GPIO.cleanup()
    cam.release()
    print("Program terminated")
